<a href="https://colab.research.google.com/github/aalokhya11/NLP_LAB/blob/main/NLP_l31_ASSIGNMENT_8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

ASSIGNMENT-8

Course Name: Natural Language Processing

Date: 2/09/2024

1. Example text data (you can replace this with any larger corpus) text = """ Once upon a time, there was a little girl named Red Riding Hood. She loved to visit her grandmother, who lived in the woods. One day, her mother asked her to take a basket of goodies to her grandmother. On her way through the woods, she met a big bad wolf who wanted to eat her. [CO5]

(i) Build the Transformer Model on above dataset

In [3]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import string

# Preprocess the text data (simple tokenization)
text = """Once upon a time, there was a little girl named Red Riding Hood. She loved to visit her grandmother..."""
text = text.lower().translate(str.maketrans('', '', string.punctuation))
vocab = sorted(set(text))
vocab_size = len(vocab)
char_to_idx = {char: idx for idx, char in enumerate(vocab)}
idx_to_char = {idx: char for idx, char in enumerate(vocab)}

# Simple Dataset for character-level modeling
class TextDataset(Dataset):
    def __init__(self, text, seq_length):
        self.text = text
        self.seq_length = seq_length

    def __len__(self):
        return len(self.text) - self.seq_length

    def __getitem__(self, idx):
        seq = self.text[idx:idx + self.seq_length]
        target = self.text[idx + 1:idx + self.seq_length + 1]
        seq = torch.tensor([char_to_idx[c] for c in seq])
        target = torch.tensor([char_to_idx[c] for c in target])
        return seq, target

# Parameters
seq_length = 25
dataset = TextDataset(text, seq_length)
dataloader = DataLoader(dataset, batch_size=32, shuffle=True)

# Simple Transformer Model
class TransformerModel(nn.Module):
    def __init__(self, vocab_size, embed_size, num_heads, hidden_dim, num_layers):
        super(TransformerModel, self).__init__()
        self.embedding = nn.Embedding(vocab_size, embed_size)
        self.pos_encoder = nn.TransformerEncoderLayer(embed_size, num_heads, hidden_dim)
        self.transformer = nn.TransformerEncoder(self.pos_encoder, num_layers)
        self.fc = nn.Linear(embed_size, vocab_size)

    def forward(self, src):
        embedded = self.embedding(src)
        transformer_output = self.transformer(embedded)
        output = self.fc(transformer_output)
        return output

# Instantiate the model
embed_size = 128
num_heads = 4
hidden_dim = 256
num_layers = 2
model = TransformerModel(vocab_size, embed_size, num_heads, hidden_dim, num_layers)


(ii) Train the model using 20, 60, 70 epochs

In [6]:
# Training function
def train_model(model, epochs):
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=0.001)

    model.train()
    for epoch in range(epochs):
        total_loss = 0
        for batch, (seq, target) in enumerate(dataloader):
            optimizer.zero_grad()
            output = model(seq)
            loss = criterion(output.view(-1, vocab_size), target.view(-1))
            loss.backward()
            optimizer.step()
            total_loss += loss.item()
        print(f'Epoch {epoch + 1}/{epochs}, Loss: {total_loss / len(dataloader)}')

# Train the model for 20 epochs
train_model(model, 20)
# train_model(model, 60)
train_model(model, 60)
# train_model(model, 70)
train_model(model, 70)


Epoch 1/20, Loss: 1.4699089129765828
Epoch 2/20, Loss: 1.496749758720398
Epoch 3/20, Loss: 1.4715338945388794
Epoch 4/20, Loss: 1.4810562133789062
Epoch 5/20, Loss: 1.4541263182957966
Epoch 6/20, Loss: 1.48347802956899
Epoch 7/20, Loss: 1.479983965555827
Epoch 8/20, Loss: 1.4450130065282185
Epoch 9/20, Loss: 1.4387729962666829
Epoch 10/20, Loss: 1.4853559335072835
Epoch 11/20, Loss: 1.4602420330047607
Epoch 12/20, Loss: 1.4423662821451824
Epoch 13/20, Loss: 1.4305086533228557
Epoch 14/20, Loss: 1.4286735852559407
Epoch 15/20, Loss: 1.4580808877944946
Epoch 16/20, Loss: 1.4862746000289917
Epoch 17/20, Loss: 1.4593278964360554
Epoch 18/20, Loss: 1.4344836473464966
Epoch 19/20, Loss: 1.464647889137268
Epoch 20/20, Loss: 1.456692377726237
Epoch 1/60, Loss: 1.4835400183995564
Epoch 2/60, Loss: 1.4401435852050781
Epoch 3/60, Loss: 1.4280763069788616
Epoch 4/60, Loss: 1.5100206136703491
Epoch 5/60, Loss: 1.4657132625579834
Epoch 6/60, Loss: 1.4608285427093506
Epoch 7/60, Loss: 1.4477972984313

 (iii) After training, use the model to generate new text by feeding it an initial seed text



In [7]:
# Function to generate text
def generate_text(model, seed_text, max_length):
    model.eval()
    input_seq = torch.tensor([char_to_idx[c] for c in seed_text]).unsqueeze(0)
    generated_text = seed_text

    for _ in range(max_length):
        with torch.no_grad():
            output = model(input_seq)
            next_char_idx = torch.argmax(output[:, -1, :], dim=-1).item()
            next_char = idx_to_char[next_char_idx]
            generated_text += next_char
            input_seq = torch.cat([input_seq, torch.tensor([[next_char_idx]])], dim=1)

    return generated_text

# Generate text using a seed text
seed_text = "once upon a time"
generated = generate_text(model, seed_text, 100)
print("Generated Text:\n", generated)


Generated Text:
 once upon a time re re re re re re re re re re re re re re re re re re re re re re re re re re re re re re re re re 


(iv) Experimenting and Improving the Model by large dataset and hyper tune parameter.

In [8]:
# Adjust model parameters for hyperparameter tuning
embed_size = 256  # Increase embedding size
num_heads = 8     # Increase number of attention heads
hidden_dim = 512  # Increase hidden dimension
num_layers = 4    # Increase number of layers

# Reinitialize the model with new parameters
model = TransformerModel(vocab_size, embed_size, num_heads, hidden_dim, num_layers)

# Load a larger dataset or continue training on the current dataset
# Train the model again with tuned hyperparameters
train_model(model, 70)  # Example with 70 epochs


/usr/local/lib/python3.10/dist-packages/torch/nn/modules/transformer.py:307: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(f"enable_nested_tensor is True, but self.use_nested_tensor is False because {why_not_sparsity_fast_path}")


Epoch 1/70, Loss: 2.845517635345459
Epoch 2/70, Loss: 2.05670436223348
Epoch 3/70, Loss: 1.6638998985290527
Epoch 4/70, Loss: 1.57102104028066
Epoch 5/70, Loss: 1.5256163676579793
Epoch 6/70, Loss: 1.4968547423680623
Epoch 7/70, Loss: 1.4869693915049236
Epoch 8/70, Loss: 1.4893366893132527
Epoch 9/70, Loss: 1.4566171169281006
Epoch 10/70, Loss: 1.477730353673299
Epoch 11/70, Loss: 1.459456443786621
Epoch 12/70, Loss: 1.4716459115346272
Epoch 13/70, Loss: 1.4559752941131592
Epoch 14/70, Loss: 1.4556833108266194
Epoch 15/70, Loss: 1.4658392270406086
Epoch 16/70, Loss: 1.4618924458821614
Epoch 17/70, Loss: 1.4506548245747883
Epoch 18/70, Loss: 1.4678389231363933
Epoch 19/70, Loss: 1.4634709755579631
Epoch 20/70, Loss: 1.45652969678243
Epoch 21/70, Loss: 1.4480664332707722
Epoch 22/70, Loss: 1.4487045208613079
Epoch 23/70, Loss: 1.440647006034851
Epoch 24/70, Loss: 1.4519389867782593
Epoch 25/70, Loss: 1.4688206911087036
Epoch 26/70, Loss: 1.484317660331726
Epoch 27/70, Loss: 1.45653752485